In [42]:
from sklearn.datasets import load_breast_cancer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F


class mydata(Dataset):
    def __init__(self, features, targets):
        self.x_data = torch.tensor(features, dtype=torch.float32)
        self.y_data = torch.tensor(targets, dtype=torch.float32).view(-1, 1)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return len(self.x_data)


class BinaryModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BinaryModel, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return F.sigmoid(self.linear(x))
        # return self.linear(x)


data = load_breast_cancer()
# print(len(data.data[0]))
# print(data.target)

x_data = data.data
y_data = data.target

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.1, random_state=42
)

train_data = mydata(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

test_data = mydata(x_test, y_test)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

print(len(x_data[0]))

# input_dim = len(x_data[0])
# output_dim = 1

# Model setup
input_dim = x_train.shape[1]
output_dim = 1
model = BinaryModel(input_dim, output_dim)

# criterion = torch.nn.MSELoss()
criterion = torch.nn.BCELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

epochs = 1001
loss_list = []
accuracy = []

for epoch in range(epochs):
    loss_sum = 0
    for inputs, targets in train_loader:
        y_pred = model(inputs)
        loss = criterion(y_pred, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    if epoch % 100 == 0:
        print(f"epoch : {epoch}, loss : { loss_sum / len(train_loader)}")

30
epoch : 0, loss : 1808.8575973510742


KeyboardInterrupt: 

In [41]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split


# Dataset class
class MyData(Dataset):
    def __init__(self, features, targets):
        self.x_data = torch.tensor(features, dtype=torch.float32)
        self.y_data = torch.tensor(targets, dtype=torch.float32).view(-1, 1)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return len(self.x_data)


# Model class
class BinaryModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BinaryModel, self).__init__()
        self.linear1 = torch.nn.Linear(30, 16)
        self.linear2 = torch.nn.Linear(16, 8)
        self.linear3 = torch.nn.Linear(8, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.sigmoid(x)

        x = self.linear2(x)
        x = self.sigmoid(x)

        x = self.linear3(x)
        x = self.sigmoid(x)  # Sigmoid to produce output in the range [0, 1]

    # Linear output; sigmoid applied later for better numerical stability


# Load the dataset
data = load_breast_cancer()
x_data = data.data
y_data = data.target

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.1, random_state=42
)

# Create datasets and dataloaders
train_data = MyData(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

test_data = MyData(x_test, y_test)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

# Model setup
input_dim = x_train.shape[1]
output_dim = 1
model = BinaryModel(input_dim, output_dim)

# Define loss function and optimizer
criterion = torch.nn.BCEWithLogitsLoss()  # More suitable for binary classification
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop
epochs = 1001
for epoch in range(epochs):
    loss_sum = 0
    correct = 0
    total = 0
    for inputs, targets in train_loader:
        y_pred = model(inputs)
        loss = criterion(y_pred, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

        # Calculate accuracy
        predicted = torch.sigmoid(y_pred)  # Apply sigmoid to get probabilities
        predicted = (predicted > 0.5).float()  # Convert probabilities to 0 or 1
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

    accuracy = correct / total
    if epoch % 100 == 0:
        print(
            f"Epoch: {epoch}, Loss: {loss_sum / len(train_loader)}, Accuracy: {accuracy * 100:.2f}%"
        )

print("Training complete.")

AttributeError: 'NoneType' object has no attribute 'size'